# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

'pytor'

# Result Filename

In [2]:
result_filename='Results_vox2.csv'

# Imports

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
import sys
import time
from PIL import Image
import csv
from time import sleep
from glob import glob

## API Setting 

In [4]:
subscription_key = "4d673dca4fbb43ab8906a7ae5691c866"
endpoint="https://advdf.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
subscription_key = "2b4d12d9d44940bbbd8c48c8f3c0e575"
endpoint="https://advdf-similarity.cognitiveservices.azure.com/"
face_client=FaceClient(endpoint, CognitiveServicesCredentials(subscription_key))

## VoxCeleb 2

### Describe an Image

In [5]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.'Scarlett Johansson_fake.png'
'''
def describe_image(filename):
#     print("===== Describe an image - local =====")
    # Call API
    image= open(filename,"rb")
    description_results = computervision_client.describe_image_in_stream(image)

    # Get the captions (descriptions) from the response, with confidence level
#     print("Description of local image: ")
    if (len(description_results.captions) == 0):
#         print("No description detected.")
        return "No description detected."
    else:
        for caption in description_results.captions:
#             print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            return "'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100)

### Detect Domain-specific Content

In [6]:
'''
Detect Domain-specific Content - local
This example detects celebrites and landmarks in local images.
'''
def domain_specific_content(filename):
#     print("===== Detect Domain-specific Content - local =====")
    # Image of one or more celebrities
    image= open(filename,"rb")
    # image = image.read('Scarlett.png')

    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities", image)
    # Print detection results with name
#     print("Celebrities in the local image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
        return "",""
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
#             print(celeb["name"])
#             print(celeb["confidence"])
            return celeb["name"],celeb["confidence"]

In [7]:
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].split("_")[0]

In [8]:
def check_similarity(src_image, tar_image):
    # Detect a face in an image that contains a single face
    src_image_name = os.path.basename(src_image)
    detected_faces = face_client.face.detect_with_stream(open(src_image,"rb"))
    if not detected_faces:
        print('No face detected from image {}'.format(single_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(single_image_name))

    # Display the detected face ID in the first single-face image.
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
#     print('Detected face ID from', single_image_name, ':')
#     for face in detected_faces: 
#         print (face.face_id)
#     print()

    # Save this ID for use in Find Similar
    first_image_face_ID = detected_faces[0].face_id
    
    multi_image_name = os.path.basename(tar_image)
    detected_faces2 = face_client.face.detect_with_stream(open(tar_image,"rb"))
    if not detected_faces2:
        print('No face detected from image {}'.format(multi_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(multi_image_name))
    
    # Search through faces detected in group image for the single face from first image.
    # First, create a list of the face IDs found in the second image.
    second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
    
    if not similar_faces:
        print('No similar faces found in', multi_image_name, '.')
        return 0
    else:
        # Print the details of the similar faces detected
#         print('Similar faces found in', multi_image_name + ':')
        for face in similar_faces:
        #     print(face)
            return face.confidence
#             print('  Confidence:', face.confidence)
#             first_image_face_ID = face.face_id
#             # The similar face IDs of the single face image and the group image do not need to match, 
#             # they are only used for identification purposes in each image.
#             # The similar faces are matched using the Cognitive Services algorithm in find_similar().
#             face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
#             if face_info:
#                 print('  Face ID: ', first_image_face_ID)
#                 print('  Face rectangle:')
#                 print('    Left: ', str(face_info.face_rectangle.left))
#                 print('    Top: ', str(face_info.face_rectangle.top))
#                 print('    Width: ', str(face_info.face_rectangle.width))
#                 print('    Height: ', str(face_info.face_rectangle.height))

In [9]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity","Src Des", "Tar Des"])
file.close()
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/*")
count=0
for actor in actors_list:
    count+=1
    print(os.path.basename(actor))
    real_image=os.path.join("/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/real",
                                os.path.basename(actor)+".png")
    fake_images_list=glob(os.path.join(actor,"*/*/*"))
    print(real_image)
    result_real_pred,result_real_conf=domain_specific_content(real_image)
    real_des=describe_image(real_image)
    src=get_name(real_image)
    for fake_image in fake_images_list:
        print(fake_image)
#         continue
        result_fake_pred,result_fake_conf=domain_specific_content(fake_image)
        fake_des=describe_image(fake_image)
        tar=get_name(fake_image)
        similarity=check_similarity(real_image,fake_image)
#             print(fake_image,src,tar,
#                   result_real_pred,result_real_conf,
#                   result_fake_pred,result_fake_conf,
#                   similarity,real_des,fake_des)
        file=open(result_filename, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow([fake_image,src,tar,
              result_real_pred,result_real_conf,
              result_fake_pred,result_fake_conf,
              similarity,real_des,fake_des])
        file.close()
#             sleep(10)
#             break
#         break
#     break
# file.close()
print("Total:",count)

David Spade
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/real/David Spade.png
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David Spade/pjklCsDxG60/00382/00001.jpg
No celebrities detected.
No similar faces found in 00001.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David Spade/pjklCsDxG60/00379/00008.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David Spade/pjklCsDxG60/00379/00033.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David Spade/pjklCsDxG60/00379/00058.jpg
No similar faces found in 00058.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David Spade/pjklCsDxG60/00379/00083.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David Spade/pjklCsDxG60/00379/00108.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/David 

No celebrities detected.
No similar faces found in 00017.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Fawad Afzal Khan/joZLV0SUzfE/00108/00092.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Fawad Afzal Khan/joZLV0SUzfE/00108/00042.jpg
No celebrities detected.
No similar faces found in 00042.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Fawad Afzal Khan/joZLV0SUzfE/00108/00067.jpg
No celebrities detected.
Jordan Henderson
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/real/Jordan Henderson.png
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Jordan Henderson/JPMZAOGGHh8/00078/00026.jpg
No similar faces found in 00026.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Jordan Henderson/JPMZAOGGHh8/00078/00001.jpg
No similar faces found in 00001.jpg .
/home/sha/project/adversarial_deepf

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Emmy Rossum/ECjj4AL9Njw/00077/00139.jpg
No celebrities detected.
No similar faces found in 00139.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Emmy Rossum/ECjj4AL9Njw/00077/00014.jpg
No celebrities detected.
No similar faces found in 00014.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Emmy Rossum/ECjj4AL9Njw/00077/00239.jpg
No similar faces found in 00239.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Emmy Rossum/ECjj4AL9Njw/00077/00189.jpg
No celebrities detected.
No similar faces found in 00189.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Emmy Rossum/ECjj4AL9Njw/00077/00039.jpg
No similar faces found in 00039.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Emmy Rossum/ECjj4AL9Njw/00077/00114.jpg
No celebrities detected.
/home/sha/project/ad

No similar faces found in 00226.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elisabeth Hasselbeck/GbE1KUG6IUM/00025/00051.jpg
No celebrities detected.
No similar faces found in 00051.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elisabeth Hasselbeck/GbE1KUG6IUM/00025/00176.jpg
No similar faces found in 00176.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elisabeth Hasselbeck/GbE1KUG6IUM/00025/00076.jpg
No celebrities detected.
No similar faces found in 00076.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elisabeth Hasselbeck/GbE1KUG6IUM/00025/00201.jpg
No similar faces found in 00201.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elisabeth Hasselbeck/GbE1KUG6IUM/00025/00101.jpg
No similar faces found in 00101.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elisabeth Hasselbeck/GbE1KU

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00249/00033.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00249/00058.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00249/00083.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00249/00108.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00249/00133.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00245/00160.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Justin Bieber/qLNvRwMkhik/00245/00135.jpg
No celebrities detected.
No face detected from image 00135.jpg
Trey Songz
/home/sha/project/adversarial_de

...

No celebrities detected.
No similar faces found in 00373.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elizabeth Shannon/0Ocu8l1eAng/00003/00023.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elizabeth Shannon/0Ocu8l1eAng/00003/00048.jpg
No celebrities detected.
No similar faces found in 00048.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elizabeth Shannon/0Ocu8l1eAng/00003/00098.jpg
No celebrities detected.
No similar faces found in 00098.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elizabeth Shannon/0Ocu8l1eAng/00003/00498.jpg
No celebrities detected.
No similar faces found in 00498.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Elizabeth Shannon/0Ocu8l1eAng/00003/00073.jpg
No celebrities detected.
No similar faces found in 00073.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/S

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00484/00067.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00485/00250.jpg
No celebrities detected.
No similar faces found in 00250.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00485/00025.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00485/00150.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00485/00100.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00485/00075.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Katie Price/xx-XbCSKh34/00485/00350.jpg
No celebrities detected.
No similar faces found in 00350.jpg .
/home/sha/project/adversarial_deepfake/src/S

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00085/00025.jpg
No celebrities detected.
No similar faces found in 00025.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00085/00150.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00085/00100.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00085/00075.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00085/00050.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00085/00175.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/Britney Spears/ApUy5iGR3Yc/00

No celebrities detected.
No similar faces found in 00050.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/LeBron James/Z0DjeR0jKFs/00281/00000.jpg
No celebrities detected.
No similar faces found in 00000.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/LeBron James/Z0DjeR0jKFs/00286/00001.jpg
No celebrities detected.
No similar faces found in 00001.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/LeBron James/Z0DjeR0jKFs/00280/00092.jpg
No celebrities detected.
No similar faces found in 00092.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/LeBron James/Z0DjeR0jKFs/00280/00067.jpg
No celebrities detected.
No similar faces found in 00067.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox2/fake/LeBron James/Z0DjeR0jKFs/00285/00074.jpg
No celebrities detected.
No similar faces found in 00074.jpg .
/home/sha/project/adversarial_deepfake/src

Total: 50
